# Scrape

In [1]:
from bs4 import BeautifulSoup
import requests
import polars as pl
import re

league_name = 'nemelee'
nemelee_tournaments_url = f'https://braacket.com/league/{league_name}/tournament?rows=200'

In [2]:
tournament_url = 'https://braacket.com/tournament/765DFCA4-947A-4218-A1F8-3EBF3E9CDCBE/match'

# Define scraper components

In [3]:
def scrape_tournament_tags(url):
    response = requests.get(url)
    tournament_soup = BeautifulSoup(response.content, 'html.parser')
    num_pages = (
        tournament_soup
        .find(class_='search-pagination')
        .find_all(class_='input-group-addon')[-1]
        .text.split(' ')[-1].strip()
    )

    tournament_tags = []
    for i in range(1, int(num_pages) + 1):
        url = f'{nemelee_tournaments_url}&page={i}'
        response = requests.get(url)
        tournament_soup = BeautifulSoup(response.content, 'html.parser')
        tournament_tags += list(
            map(
                lambda x: x.find_parent(class_='panel'),
                tournament_soup.find_all(string='Detail')
            )
        )
    return tournament_tags


In [4]:
tournament_tags = scrape_tournament_tags(nemelee_tournaments_url)

In [57]:
tournament_tags[0].find(class_='country_flag').find('img').get('src', None)

'/assets/images/country/flag/us.png'

In [5]:
def extract_tournament_data(tournament_tags):
    tournaments = []
    for tournament_tag in tournament_tags:
        data = {
            'url': tournament_tag.find(class_='panel-heading').find('a')['href'],
            'name': tournament_tag.find(class_='panel-heading').find('a').text.strip(),
            'date': tournament_tag.find(string='Date').parent
                     .find_next_sibling().text.strip() if tournament_tag.find(string='Date').parent.find_next_sibling() else None,
            'country': (tournament_tag.find(class_='country_flag').find('img').get('src', None)
                        if tournament_tag.find(class_='country_flag') else None),
            'region': (tournament_tag.find(class_='country_region_flag').get('src', None)
                       if tournament_tag.find(class_='country_region_flag') else None),
            'number_of_players': (int(tournament_tag.find(data_original_title_='Imported players').text.strip())
                                  if tournament_tag.find(data_original_title_='Imported players') else None)
        }
        tournaments.append(data)
    return tournaments


In [ ]:
tournaments = extract_tournament_data(tournament_tags)
# Convert the list of dictionaries to a Polars DataFrame
tournaments_df = pl.DataFrame(tournaments, schema={col: pl.String for col in tournaments[0].keys()}).filter(pl.col('url').is_not_null())
# Save the DataFrame to a CSV file
pl.config.set_tbl_rows(2)
tournaments_df

In [ ]:
def scrape_stage_urls(match_url):
    response = requests.get(match_url)
    match_soup = BeautifulSoup(response.content, 'html.parser')
    stage_urls = list(set(
        map(
            lambda x: x['href'],
            match_soup.find_all(attrs={"href": re.compile(r"^/tournament/.*/stage/.*$")}),
        )
    ))
    return stage_urls


In [ ]:
match_url = f'https://braacket.com/{tournaments[0]['url']}/match'
print(match_url)
stage_urls = scrape_stage_urls(match_url)

In [ ]:
tournament_url = 'https://braacket.com/tournament/765DFCA4-947A-4218-A1F8-3EBF3E9CDCBE/match'
scrape_stage_urls(tournament_url)

In [7]:
player_url = '/tournament/633C3E6E-A97A-45F9-B1B4-1ADA348C350C/player/F6E88BFB-8602-4A5A-9A02-7FA6D3098A6C'

def scrape_player_url(player_url):
    response = requests.get(f'https://braacket.com/{player_url}')
    player_soup = BeautifulSoup(response.content, 'html.parser')
    league_links = player_soup.find_all(attrs={"href": re.compile(rf"^/league/{league_name}/player/.*$")})
    if not league_links:
        return None
    player_url = league_links[0]['href']
    return player_url
scrape_player_url(player_url)

'/league/nemelee/player/C2069576-DB5B-42E0-9C4B-31182802A60F?'

In [8]:
def scrape_stage_matches(tournament: dict, stage_url: str):
    matches = []
    response = requests.get(f'https://braacket.com/{stage_url}')
    stage_soup = BeautifulSoup(response.content, 'html.parser')

    player_url_dict = {}
    stage_encounters = stage_soup.find_all(class_='tournament_encounter-row')
    for encounter in stage_encounters:
        winner_tournament_url = encounter.select('.tournament_encounter_opponent.winner a')[0]['href']
        loser_tournament_url = encounter.select('.tournament_encounter_opponent.loser a')[0]['href']
        if winner_tournament_url not in player_url_dict:
            player_url_dict[winner_tournament_url] = scrape_player_url(winner_tournament_url)
        if loser_tournament_url not in player_url_dict:
            player_url_dict[loser_tournament_url] = scrape_player_url(loser_tournament_url)
        data = {
            'encounter_id': encounter.find(class_='tournament_encounter-id').text.strip(),
            'winner_url': player_url_dict[winner_tournament_url],
            'winner': encounter.select('.tournament_encounter_opponent.winner')[0].text.strip(),
            'loser_url': player_url_dict[loser_tournament_url],
            'loser': encounter.select('.tournament_encounter_opponent.loser')[0].text.strip(),
            'winner_score': encounter.select('.tournament_encounter-score.winner')[0].text.strip(),
            'loser_score': encounter.select('.tournament_encounter-score.loser')[0].text.strip(),
        }
        data.update({
            f'tournament_{k}': v
            for k, v in tournament.items()
        })
        matches.append(data)
    return matches


#matches = scrape_stage_matches(tournaments[0], stage_urls[0])

In [ ]:
# Convert the list of dictionaries to a Polars DataFrame
matches_df = (
    pl.DataFrame(matches, schema={col: pl.String for col in matches[0].keys()})
)
# Save the DataFrame to a CSV file
matches_df

In [78]:
response = requests.get(f'https://braacket.com/{stage_urls[0]}')
stage_soup = BeautifulSoup(response.content, 'html.parser')

stage_encounters = stage_soup.find_all(class_='tournament_encounter-row')


In [79]:
encounter = stage_encounters[0]
data = {
    'encounter_id': encounter.find(class_='tournament_encounter-id').text.strip(),
    'winner_url': encounter.select('.tournament_encounter_opponent.winner a')[0]['href'],
    'winner': encounter.select('.tournament_encounter_opponent.winner')[0].text.strip(),
    'loser_url': encounter.select('.tournament_encounter_opponent.loser a')[0]['href'],
    'loser': encounter.select('.tournament_encounter_opponent.loser')[0].text.strip(),
    'winner_score': encounter.select('.tournament_encounter-score.winner')[0].text.strip(),
    'loser_score': encounter.select('.tournament_encounter-score.loser')[0].text.strip(),
}
data

{'encounter_id': '1',
 'winner_url': '/tournament/633C3E6E-A97A-45F9-B1B4-1ADA348C350C/player/F6E88BFB-8602-4A5A-9A02-7FA6D3098A6C',
 'winner': 'MATE | Kalvar',
 'loser_url': '/tournament/633C3E6E-A97A-45F9-B1B4-1ADA348C350C/player/81101D13-20BA-4EAE-B751-833DA282F716',
 'loser': 'kuro',
 'winner_score': '3',
 'loser_score': '0'}

In [80]:
encounter.select('.tournament_encounter_opponent.loser a')

[<a href="/tournament/633C3E6E-A97A-45F9-B1B4-1ADA348C350C/player/81101D13-20BA-4EAE-B751-833DA282F716">kuro</a>]

## Run Scraper

In [84]:
tournament_tags = scrape_tournament_tags(nemelee_tournaments_url)

In [85]:
tournaments = extract_tournament_data(tournament_tags)
tournaments_df = pl.DataFrame(tournaments, schema={col: pl.String for col in tournaments[0].keys()}).filter(pl.col('url').is_not_null())
tournaments_df.write_csv('data/tournaments.csv')

matches = []
for tournament in tournaments:
    tournament_matches = []
    try:
        match_url = f'https://braacket.com/{tournament["url"]}/match'
        stage_urls = scrape_stage_urls(match_url)
        for stage_url in stage_urls:
            tournament_matches += scrape_stage_matches(tournament, stage_url)
    except Exception as e:
        print(f"Error processing tournament {tournament['name']}, {tournament['url']}: {e}")
        continue
    matches += tournament_matches


Error processing tournament FreePlay Melee #104, /tournament/5DBE0AC6-EBEB-4C2E-B64A-F8F432F85EF8: list index out of range
Error processing tournament New Game Plus Revival 7.7, /tournament/765DFCA4-947A-4218-A1F8-3EBF3E9CDCBE: list index out of range
Error processing tournament Melee Monday @ Bumper 9, /tournament/CADFAF5F-9AEE-4071-BE1E-977E41C05085: list index out of range
Error processing tournament Buffalo Wild Wednesdays v5.1.3, /tournament/9651E764-012D-4652-B7D7-9B81BC56B081: list index out of range
Error processing tournament The 2024 Upstate Melee Championship & Arcadian [NE Only], /tournament/6F173939-0D68-46C0-BAB3-B0392200CE1F: list index out of range
Error processing tournament One Up Melee 9.4.24, /tournament/343299CA-F3AE-47B1-9899-E6E0E1B17DFF: list index out of range
Error processing tournament One Up Melee 8.7.24, /tournament/AF7BAD62-EBED-461E-BC34-04A47B1F7B48: list index out of range
Error processing tournament New Game Plus Revival 6.9, /tournament/8DB1E88B-E0BF-

KeyboardInterrupt: 

In [13]:
match_url = 'https://braacket.com/tournament/5DBE0AC6-EBEB-4C2E-B64A-F8F432F85EF8/match'
tournament_matches = []
stage_urls = scrape_stage_urls(match_url)
for stage_url in stage_urls:
    tournament_matches += scrape_stage_matches(stage_url)

ConnectionError: HTTPSConnectionPool(host='braacket.com', port=443): Max retries exceeded with url: /tournament/5DBE0AC6-EBEB-4C2E-B64A-F8F432F85EF8/match (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x74503654fb10>: Failed to resolve 'braacket.com' ([Errno -2] Name or service not known)"))

In [11]:
# Only scrape matches from tournaments not already in matches
matches_df = pl.read_csv('data/matches.csv')

tournaments = extract_tournament_data(tournament_tags)
tournaments_df = pl.DataFrame(tournaments, schema={col: pl.String for col in tournaments[0].keys()}).filter(pl.col('url').is_not_null())
tournaments_df.write_csv('data/tournaments.csv')

matches = []
for tournament in tournaments:
    if matches_df.select((pl.col('tournament_url') == tournament['url']).any()).item():
        continue
    print(f"Scraping matches for tournament {tournament['name']}, {tournament['url']}")
    tournament_matches = []
    try:
        match_url = f'https://braacket.com/{tournament["url"]}/match'
        stage_urls = scrape_stage_urls(match_url)
        for stage_url in stage_urls:
            tournament_matches += scrape_stage_matches(tournament, stage_url)
    except Exception as e:
        print(f"Error processing tournament {tournament['name']}, {tournament['url']}: {e}")
        continue
    matches += tournament_matches

Scraping matches for tournament New Game Plus Revival 8.3, /tournament/EBFBECF8-08C9-4962-8EA1-53039F6048A7
Scraping matches for tournament FreePlay Melee #114, /tournament/271B645C-277C-48E8-ADD9-7B66683698AC
Scraping matches for tournament Pho Tai Melee #97 OwO, /tournament/A75B8B35-2596-464F-8F26-35B5889CEB5B
Scraping matches for tournament FreePlay Melee #104, /tournament/5DBE0AC6-EBEB-4C2E-B64A-F8F432F85EF8
Scraping matches for tournament New Game Plus Revival 7.7, /tournament/765DFCA4-947A-4218-A1F8-3EBF3E9CDCBE
Scraping matches for tournament FreePlay Melee #98, /tournament/A812E6CC-BB8F-447E-A868-672565EEDF59
Scraping matches for tournament Melee Monday @ Bumper’s 12, /tournament/19F24CF6-D9D0-4615-869D-09B637E364E6
Scraping matches for tournament Return to Vacationland 7, /tournament/E237231B-7841-4665-BE3E-207D170BA6EB
Scraping matches for tournament SSS 20.2 - A New Hampshire Melee Monthly!, /tournament/69327137-A2AB-4549-BEC3-90FC65436081
Scraping matches for tournament Pho

KeyboardInterrupt: 

In [15]:
matches_df = (
    pl.DataFrame(matches, schema={col: pl.String for col in matches[0].keys()})
    # Parse date from format like '01 April 2018'
    .with_columns(pl.col('tournament_date').str.strptime(pl.Date, format='%d %B %Y'))
    .sort('encounter_id', descending=False)
    .sort('tournament_date')
)
#matches_df.write_csv('data/matches.csv')
pl.Config(tbl_rows=2)
matches_df

encounter_id,winner_url,winner,loser_url,loser,winner_score,loser_score,tournament_url,tournament_name,tournament_date,tournament_country,tournament_region,tournament_number_of_players
str,str,str,str,str,str,str,str,str,date,str,str,str
"""1""","""/league/nemelee/player/C9B8492…","""bonfire10""","""/league/nemelee/player/BAB54F3…","""Electroman""","""3""","""0""","""/tournament/FEC28155-A4E2-40EF…","""The Function 2 [NE Only]""",2022-05-07,"""/assets/images/country/flag/us…","""/assets/images/country/regions…",null
…,…,…,…,…,…,…,…,…,…,…,…,…
"""94""","""/league/nemelee/player/BAB54F3…","""OUG | Electroman""","""/league/nemelee/player/0CDB11B…","""hc | kraft""","""3""","""1""","""/tournament/633C3E6E-A97A-45F9…","""Mass Madness 50 - 10th Anniver…",2025-05-03,"""/assets/images/country/flag/us…","""/assets/images/country/regions…",null


## Create player dataframe

In [18]:
matches_df = pl.read_csv('data/matches.csv')

In [35]:
players = (
    matches_df
    .select(pl.col('winner').alias('tag'), pl.col('winner_url').alias('url'), 'tournament_date')
    .vstack(
        matches_df
        .select(pl.col('loser').alias('tag'), pl.col('loser_url').alias('url'), 'tournament_date')
    )
    .unique()
    .sort('tournament_date', descending=False)
    .group_by('url')
    .last()
    .select('tag', 'url')
)
players.write_csv('data/players.csv')
pl.Config(tbl_rows=2)
players

tag,url
str,str
"""Tank""","""/league/nemelee/player/02FCD0C…"
…,…
"""Lisa""","""/league/nemelee/player/1615412…"


# Calculate trueskill

In [36]:
import trueskill
import polars as pl

In [37]:
matches_df = pl.read_csv('data/matches.csv')
pl.Config(tbl_rows=2)
matches_df

encounter_id,winner_url,winner,loser_url,loser,winner_score,loser_score,tournament_url,tournament_name,tournament_date,tournament_country,tournament_region,tournament_number_of_players
i64,str,str,str,str,i64,i64,str,str,str,str,str,str
59,"""/league/nemelee/player/DFEBBBB…","""Nibbity""","""/league/nemelee/player/733440B…","""OverWorld | Cinema""",3,1,"""/tournament/B25C5099-BAE3-44B1…","""HoG: Phoenix 3.7""","""2022-11-10""",null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…
46,"""/league/nemelee/player/4934D40…","""stocmoney""","""/league/nemelee/player/826CE4C…","""Velma""",2,0,"""/tournament/76086F2C-43A5-4E12…","""New Game Plus Revival 2.9""","""2022-07-19""",null,null,null


In [38]:
players_dict = {}

def rate_players(row: dict) -> dict:
    # Create TrueSkill rating objects for the players
    if players_dict.get(row['winner_url']) is None:
        players_dict[row['winner_url']] = trueskill.Rating()
    if players_dict.get(row['loser_url']) is None:
        players_dict[row['loser_url']] = trueskill.Rating()

    result = {
        'winner_rating': players_dict[row['winner_url']],
        'loser_rating': players_dict[row['loser_url']],
    }

    # Update the ratings based on the match outcome
    new_winner_rating, new_loser_rating = trueskill.rate_1vs1(
        players_dict[row['winner_url']], players_dict[row['loser_url']])
    players_dict[row['winner_url']] = new_winner_rating
    players_dict[row['loser_url']] = new_loser_rating

    result.update({
        'new_winner_rating': new_winner_rating,
        'new_loser_rating': new_loser_rating,
    })
    return result

In [39]:
players_dict = {}

matches_with_ratings = matches_df.with_columns(
    pl.struct(['winner_url', 'loser_url'])
        .map_elements(rate_players, return_dtype=pl.Struct)
        .alias('new_ratings')
).unnest('new_ratings')

## 95 CI
In TrueSkill, rating is a Gaussian distribution which starts from $\mathcal{ N }( 25, \frac{ 25 }{ 3 }^2 )$. $\mu$ is an average skill of player, and $\sigma$ is a confidence of the guessed rating. A real skill of player is between $\mu \pm 2\sigma$ with 95% confidence.

In [40]:
matches_with_ratings.write_csv('data/matches-with-ratings.csv')

## Export ranking

In [41]:
player_ratings = pl.DataFrame({
    'url': [player_url for player_url in players_dict.keys()],
    'rating': [rating for rating in players_dict.values()],
})

In [42]:
pl.Config(tbl_rows=5000)
player_ratings.join(players, on='url').sort('rating', descending=True)

url,rating,tag
str,f64,str
"""/league/nemelee/player/FDFA0C9…",47.916552,"""Nouns | Aklo"""
"""/league/nemelee/player/566A026…",47.305976,"""DTL | 2Saint"""
"""/league/nemelee/player/891E861…",47.083609,"""RedBull GG IFM | aMSa"""
"""/league/nemelee/player/37B3203…",46.809406,"""Spark"""
"""/league/nemelee/player/197387D…",45.618231,"""TLOC | Salt"""
"""/league/nemelee/player/6B2BEB6…",44.381873,"""DrLobster"""
"""/league/nemelee/player/EB30F2B…",43.937776,"""Inky"""
"""/league/nemelee/player/30D4C0A…",43.584675,"""SHDJ | Mekk"""
"""/league/nemelee/player/726A52B…",43.437617,"""YAMI"""
